In [1]:
cd /home/nhan-softzone/law_search

/home/nhan-softzone/law_search


/home/nhan-softzone/miniconda3/envs/law_agent/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
from modules.retrieval_engine.hybrid_engine import HybridRetrievalEngine
from modules.retrieval_engine.dense_engine import DenseRetrievalEngine
from modules.retrieval_engine.bm25_engine import BM25RetrievalEngine

/home/nhan-softzone/miniconda3/envs/law_agent/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import numpy as np
from tqdm import tqdm
from datasets import load_dataset
import random

In [4]:
model_name = 'bkai-foundation-models/vietnamese-bi-encoder'
dataset_name = 'another-symato/VMTEB-Zalo-legel-retrieval'
word_segment = True
num_corpus_addition = 0
engine = HybridRetrievalEngine(dense_params={'model_name': model_name, 
                                             'dimension': 768})

In [5]:
if word_segment:
    dataset_name += '-wseg'

corpus = load_dataset(dataset_name, "corpus", split = 'train')
queries = load_dataset(dataset_name, "queries", split = 'train')
relevant_docs_data = load_dataset(dataset_name, 'data_ir', split="test")
    
required_corpus_ids = list(map(str, relevant_docs_data["corpus_id"]))
if num_corpus_addition:
    required_corpus_ids += random.sample(corpus["id"], k = num_corpus_addition)
print(len(required_corpus_ids))
corpus = corpus.filter(lambda x: x["id"] in required_corpus_ids)
    
required_queries_ids = list(map(str, relevant_docs_data["query_id"]))
queries = queries.filter(lambda x: x["query_id"] in required_queries_ids)
    
corpus_dict = dict(zip(corpus["id"], corpus["text"]))
queries_dict = dict(zip(queries["query_id"], queries["question"]))
relevant_docs = {} 
for qid, corpus_ids in zip(relevant_docs_data["query_id"], relevant_docs_data["corpus_id"]):
    qid = str(qid)
    corpus_ids = str(corpus_ids)
    if qid not in relevant_docs:
        relevant_docs[qid] = set()
    relevant_docs[qid].add(corpus_ids)

640


In [6]:
corpus_content = corpus['text']

In [7]:
engine.index_documents(corpus_content)

True

In [44]:
for qid in tqdm(range(5)):
    query = queries[qid]['question']
    query_id = queries[qid]['query_id']
    top_ids, scores = engine.search(query, top_k = 10)
    relevance = [1 if corpus[int(doc_id)]['id'] in relevant_docs[query_id] else 0 for doc_id in top_ids]
    print(relevance)

  0%|          | 0/5 [00:00<?, ?it/s]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


[0, 1, 0, 0, 0, 0, 0, 0, 0, 0]


[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


 80%|████████  | 4/5 [00:00<00:00, 36.25it/s]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]


100%|██████████| 5/5 [00:00<00:00, 36.27it/s]

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
